# Set up and import tahoe geometry

In [ ]:
import pandas as pd
import numpy as np
import arcpy
from arcgis.features import FeatureLayer

service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'

feature_layer = FeatureLayer(service_url)
tahoe_geometry_fields = ['YEAR', 'STATE', 'GEOGRAPHY', 'GEOID', 'TRPAID', 'NEIGHBORHOOD']
query_result = feature_layer.query(out_fields=",".join(tahoe_geometry_fields))
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
tahoe_geometry = pd.DataFrame([feature.attributes for feature in feature_list])

county_lookup = {
    '005': 'Douglas County (Tahoe Basin)',
    '017': 'El Dorado County (Tahoe Basin)',
    '031': 'Washoe County (Tahoe Basin)',
    '061': 'Placer County (Tahoe Basin)'
}

state_lookup = {
    '32': 'Nevada',
    '06': 'California'
}

inflation_adjustment = {
    '2000': 1.57,
    '2010': 1.24
}

## Define Functions

In [ ]:
def create_or_append_df(df, summary_df):
    if df.empty:
        df = summary_df.copy()
    else:
        df = pd.concat([df, summary_df])
    return df

def calculate_median_value(df, bin_column, sort_column, count_column, category_field, category,grouping_variables, cumulative_sorting_variables):
        # Create a new DataFrame to avoid modifying the original one
    #change value to count column
    #Do we need to handle excluding non-tahoe things here or should we do it in the input?
    summary_df = df.copy()
    summary_df[count_column]=summary_df[count_column].astype(int)
    #summary_df=summary_df.loc[summary_df[count_column]!='510']
    summary_df = summary_df.groupby(grouping_variables)[count_column].sum()
    summary_df = summary_df.reset_index()
    #This handles -6666 values that they sometimes add for unknown data
    summary_df = summary_df.loc[summary_df[count_column]>=0]
    
    
    summary_df= summary_df.loc[summary_df[category_field]==category]
    # Sort the DataFrame based on the variable name column
    # This depends on the fact that census variables start with the lowest value and go up 
    #This needs to all be rethought to have some kind of window function to handle multiple years
    summary_df.sort_values(by=sort_column, inplace=True)
    summary_df = summary_df.reset_index()
    
    # Extract lower and upper limits from bin categories
    #This uses regex to find numbers and removes commas to make numbers numbers 
    pattern = r'(\d+[\d,]*)'
    summary_df['temp'] = summary_df[bin_column].str.replace(',', '').str.findall(pattern)
    #Looks for values that have two numbers and puts empty placeholders for the ones that only have one (upper and lower)
    summary_df[['Lower', 'Upper']] = summary_df['temp'].apply(lambda x: pd.Series(x[:2]) if len(x) == 2 else pd.Series([None, None]))
    summary_df['Lower'] = summary_df['Lower'].astype(float)
    summary_df['Upper'] = summary_df['Upper'].astype(float)
    # Handle first category
    
    first_upper = float(summary_df['temp'].iloc[0][0])
    low_variable_name = summary_df[bin_column].iloc[0]

    summary_df.loc[summary_df[bin_column]==low_variable_name,'Lower'] = 0  # Set lower value to 0
    summary_df.loc[summary_df[bin_column]==low_variable_name,'Upper'] = first_upper

    
    # Handle last category
    
    last_lower = float(summary_df['temp'].iloc[-1][0])
    upper_variable_name = summary_df[bin_column].iloc[-1]
    summary_df.loc[summary_df[bin_column]==upper_variable_name,'Lower'] = last_lower
    summary_df.loc[summary_df[bin_column]==upper_variable_name,'Upper'] = np.inf  # Set upper value to infinity
    summary_df[count_column]= summary_df[count_column].astype(float)   
    # Calculate cumulative count
    cumulative_grouping_variables = grouping_variables

    cumulative_grouping_variables.remove(bin_column)
    cumulative_grouping_variables.remove(sort_column)

    #Update this to be parameterized
    
    summary_df.sort_values(by=cumulative_sorting_variables, inplace=True)
    summary_df = summary_df.reset_index()
    summary_df['cumulative_sum'] = summary_df.groupby(cumulative_grouping_variables, as_index=False)[count_column].cumsum()
    summary_df['TotalSum'] = summary_df.groupby(cumulative_grouping_variables, as_index=False)[count_column].transform('sum')
    summary_df['previous_cumulative'] = summary_df['cumulative_sum'].shift()

    summary_df = summary_df.loc[summary_df['cumulative_sum']>=(summary_df['TotalSum']/2)].groupby(cumulative_grouping_variables, as_index=False).first()

    summary_df['cumulative_difference'] = summary_df['TotalSum']  / 2 - summary_df['previous_cumulative']
    summary_df['interpolation_ratio'] = summary_df['cumulative_difference'] /  (summary_df['cumulative_sum']- summary_df['previous_cumulative'])
    summary_df['median_value'] = summary_df['Lower'] + summary_df['interpolation_ratio'] * (summary_df['Upper'] - summary_df['Lower'])
    
    
    return summary_df


def categorize_values(census_df, category_csv, category_column, grouping_prefix):
    categories = pd.read_csv(category_csv)    
    census_df['value'] = census_df['value'].astype(float)
    joined_data = census_df.merge(categories, on = 'variable_code', how = 'left')
    joined_data.sort_values(by='variable_code', inplace=True)
    #This will get rid of any extra columns in the category_csv
    group_columns = [column for column in census_df if column not in ['value', 'variable_code', 'variable_name', 'MarginOfError','OBJECTID']]
    group_columns.append(category_column)
    #grouped_data = joined_data.groupby(group_columns, as_index=False)['value'].sum()    
    print(group_columns)
    grouped_data = joined_data.groupby(group_columns, as_index=False, dropna=False).agg({'value':'sum',
                                                                           'variable_code':lambda x: grouping_prefix +  ', '.join(x)})
    
    #Need to return this formatted for appending to the table - need to get locations of variable_code and variable name, 
    #add them in as columns in those locations and then populate them with category column nanme
    var_code_col_location = census_df.columns.get_loc('variable_code')
    var_name_col_location = census_df.columns.get_loc('variable_name')
    var_moe_col_location = census_df.columns.get_loc('MarginOfError')
    grouped_data.insert(var_moe_col_location, 'MarginOfError', '')
    #grouped_data.insert(var_code_col_location, 'variable_code','Grouped Value')
    grouped_data.insert(var_name_col_location, 'variable_name','')
    #grouped_data['variable_code'] = grouped_data['variable_code'] +  '_Grouped'
    grouped_data['variable_name'] = grouped_data[category_column]
    grouped_data['dataset']= grouping_prefix + grouped_data['dataset']
    grouped_data['variable_category']= grouping_prefix +  grouped_data['variable_category'] 
    columns_to_keep = [column for column in census_df if column not in ['OBJECTID']]
    grouped_data= grouped_data[columns_to_keep]
    return grouped_data


# Generate Basin Estimates

## Import data for Basin Estimates

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
all_data = pd.DataFrame([feature.attributes for feature in feature_list])

all_data_clean = all_data.loc[(all_data['county']!='510')&(all_data['value']>=0)]
all_data_clean.loc[all_data_clean['county'].isin(['005','017']),'north_south'] = 'South Lake'
all_data_clean.loc[all_data_clean['county'].isin(['031','061']),'north_south'] = 'North Lake'
all_data_clean['county_name'] = all_data_clean['county'].apply(lambda x: county_lookup.get(x, None))
all_data_clean['state_name'] = all_data_clean['state'].apply(lambda x: state_lookup.get(x, None))


all_data_clean = all_data_clean.dropna(subset='value')


## Summed Estimates

### Population Totals

In [ ]:
filtered_population_data = all_data_clean.loc[(all_data_clean['variable_name']=='Total Population')]
filtered_population_data
summarized_population_data_basin = filtered_population_data.groupby([ 'dataset', 'variable_code', 'year_sample'], as_index=False).sum(['value'])
summarized_population_data_county = filtered_population_data.groupby(['dataset', 'variable_code', 'year_sample', 'county_name'], as_index=False).sum(['value'])
summarized_population_data_north_south = filtered_population_data.groupby(['dataset', 'variable_code', 'year_sample', 'north_south'], as_index=False).sum(['value'])

In [ ]:
def sum_across_levels(df, variable_name, category_name):
    filtered_df = df.loc[(df['variable_name']==variable_name)]
    basin_summary = filtered_df.groupby([ 'dataset', 'sample_level', 'variable_name', 'variable_code', 'year_sample'], as_index=False).sum(['value'])
    county_summary = filtered_df.groupby(['dataset', 'sample_level', 'variable_name', 'variable_code', 'year_sample', 'county_name'], as_index=False).sum(['value'])
    north_south_summary = filtered_df.groupby(['dataset', 'sample_level', 'variable_name', 'variable_code', 'year_sample', 'north_south'], as_index=False).sum(['value'])
    state_summary = filtered_df.groupby(['dataset', 'sample_level', 'variable_name', 'variable_code', 'year_sample', 'state_name'], as_index=False).sum(['value'])
    #basin_summary.rename(columns = {'variable_code': 'Code', 'year_sample': 'Year'})
    basin_summary['Geography'] = 'Basin'
    county_summary['Geography'] = county_summary['county_name'] 
    north_south_summary['Geography'] = north_south_summary['north_south']
    state_summary['Geography'] = state_summary['state_name']
    columns_to_keep = ['variable_code','variable_name', 'value', 'Geography', 'year_sample', 'dataset', 'sample_level']
    basin_summary= basin_summary[columns_to_keep]
    county_summary = county_summary[columns_to_keep]
    north_south_summary = north_south_summary[columns_to_keep]
    state_summary = state_summary[columns_to_keep]
    combined_summary = pd.concat([basin_summary, county_summary, north_south_summary, state_summary], ignore_index=True)
    #if neighborhood_yn == 'Yes':
    #    neighborhood_summary = filtered_df.groupby(['dataset', 'sample_level', 'variable_name', 'variable_code', 'year_sample', 'NEIGHBORHOOD'], as_index=False).sum(['value'])
        #basin_summary.rename(columns = {'variable_code': 'Code', 'year_sample': 'Year'})
    #    neighborhood_summary['Geography'] = neighborhood_summary['NEIGHBORHOOD']
    #    combined_summary = pd.concat([combined_summary, neighborhood_summary], ignore_index=True)
    combined_summary['Category'] = category_name
    return combined_summary
def sum_multiple_variables(df, variable_list, variable_category):
    df_values=pd.DataFrame()
    for variable in variable_list:
        summed_df = sum_across_levels(df,variable, variable_category)
        df_values = create_or_append_df(df_values, summed_df)
    return df_values



## Climate Dashboard Summaries

In [ ]:
summary_variables = pd.read_csv('Census_Category_Lists/Climate_Dashboard_Age.csv')
variables = list(summary_variables['Variables'])

In [ ]:
filtered_data = all_data_clean.loc[all_data_clean['variable_category']=='TRPA 5-year Age CategoriesAge']
summed_ages = sum_multiple_variables(filtered_data, variables,'Age')
summed_ages.to_excel('summed_ages.xlsx')

In [ ]:
summed_heating = sum_across_levels(all_data_clean,'Total Heating Methods','House Heating Fuel')

In [ ]:
variables=['Renter Occupied: Two Or More Races Householder', 'Owner Occupied: Two Or More Races Householder']
df = sum_multiple_variables(all_data_clean,variables,'Tenure by Race')
df.to_excel('dashboard_fill_2.xlsx')

In [ ]:
variables=['Total Heating Methods', 'Utility gas', 'Bottled, tank, or LP gas',
           'Electricity','Fuel oil, kerosene, etc.', 'No fuel used',
           'Coal or coke','Wood','Solar energy','Other fuel']
heating_df = sum_multiple_variables(all_data_clean,variables,'House Heating Fuel')
heating_df.to_excel('heating_summaries.xlsx')

In [ ]:
variables=['Total population:  Hispanic or Latino', 'Total population:  Not Hispanic or Latino; White alone','Total population:  Not Hispanic or Latino; Black or African American alone',
           'Total population:  Not Hispanic or Latino; American Indian and Alaska Native alone', 'Total population:  Not Hispanic or Latino; Asian alone',
           'Total population:  Not Hispanic or Latino; Native Hawaiian and Other Pacific Islander alone',
           'Total population:  Not Hispanic or Latino; Some other race alone',
           'Total population:  Not Hispanic or Latino; Two or more races',
           'Total Heating Methods', 'Utility gas', 'Bottled, tank, or LP gas',
           'Electricity','Fuel oil, kerosene, etc.','Coal or coke','Wood','Solar energy','Other fuel'
           'Owner Occupied: American Indian And Alaska Native Alone Householder',
'Owner Occupied: Asian Alone Householder',
'Owner Occupied: Black Or African American Alone Householder',
'Owner Occupied: Householder 25 To 34 Years',
'Owner Occupied: Householder 35 To 44 Years',
'Owner Occupied: Householder 45 To 54 Years',
'Owner Occupied: Householder 55 To 59 Years',
'Owner Occupied: Householder 60 To 64 Years',
'Owner Occupied: Householder 65 To 74 Years',
'Owner Occupied: Householder 75 To 84 Years',
'Owner Occupied: Householder 85 Years And Over',
'Owner Occupied: Native Hawaiian And Other Pacific Islander Alone Householder',
'Owner Occupied: Some Other Race Alone Householder',
'Owner Occupied: White Alone Householder',
'Renter Occupied: American Indian And Alaska Native Alone Householder',
'Renter Occupied: Asian Alone Householder',
'Renter Occupied: Black Or African American Alone Householder',
'Renter Occupied: Householder 15 To 24 Years',
'Renter Occupied: Householder 25 To 34 Years',
'Renter Occupied: Householder 35 To 44 Years',
'Renter Occupied: Householder 45 To 54 Years',
'Renter Occupied: Householder 55 To 59 Years',
'Renter Occupied: Householder 60 To 64 Years',
'Renter Occupied: Householder 65 To 74 Years',
'Renter Occupied: Householder 75 To 84 Years',
'Renter Occupied: Householder 85 Years And Over',
'Renter Occupied: Native Hawaiian And Other Pacific Islander Alone Householder',
'Renter Occupied: Some Other Race Alone Householder',
'Renter Occupied: Total',
'Renter Occupied: White Alone Householder',
'Total: American Indian And Alaska Native Alone Householder',
'Total: Asian Alone Householder',
'Total: Black Or African American Alone Householder',
'Total: Native Hawaiian And Other Pacific Islander Alone Householder',
'Total: Some Other Race Alone Householder',
'Total: Two Or More Races Householder',
'Total: White Alone Householder'
]
race_df = sum_multiple_variables(all_data_clean,variables,'Dashboard Variables')
race_df.to_excel('Dashboard_Summary.xlsx')


In [ ]:
variables=['Owner Occupied: Householder 15 To 24 Years', 'Owner Occupied: Total', 'Renter Occupied: Two Or More Races Householder',
           'Owner Occupied: Two Or More Races Householder','Owner Occupied: American Indian And Alaska Native Alone Householder']
heating_df = sum_multiple_variables(all_data_clean,variables,'Dashboard Variables')
heating_df.to_excel('dashboard_summaries_2.xlsx')

In [ ]:
filtered_income_data = all_data_clean.loc[(all_data_clean['variable_category']=='Household Income')]
variable_name = '$200,000 or more'
category_name = 'Household Income'
rich_person_summary = sum_across_levels(all_data_clean, variable_name, category_name)
rich_person_summary.to_excel('RichPeople.xlsx')

In [ ]:
variable_name = 'Total Population'
category_name = 'Total Population'


total_population_summary = sum_across_levels(all_data_clean, variable_name, category_name)

In [ ]:
total_population_summary = total_population_summary.loc[total_population_summary['variable_code']!='P1_001N']

total_population_summary.to_excel('population_summaries.xlsx')

In [ ]:
filtered_housing_data = all_data_clean.loc[(all_data_clean['sample_level']!='block')]
variable_name = 'Total Housing Units'
category_name = 'Housing Units'

total_housing_units_summary = sum_across_levels(filtered_housing_data, variable_name, category_name)
total_housing_units_summary.to_excel('total_housing_units_summary.xlsx')

In [ ]:
filtered_housing_data = all_data_clean.loc[(all_data_clean['sample_level']!='block')]
#variable_name = 'Total Housing Units'
category_name = 'Housing Units'

variable_list= ['Total Housing Units', 'Total Housing Units: Occupied', 
    'Total Housing Units: Vacant', 'Vacant Housing Units: Seasonal, recreational, or occasional use',
    'Occupied Housing Units: Owner Occupied', 'Occupied Housing Units: Renter Occupied','Total Population']
housing_unit_summary=pd.DataFrame()
for variable in variable_list:
    df = sum_across_levels(filtered_housing_data, variable, category_name)
    housing_unit_summary = create_or_append_df(housing_unit_summary, df)

housing_unit_summary.to_excel('housing_unit_summaries.xlsx')


## 2022 Summed estimates

In [ ]:
all_data_clean_2022 = all_data_clean.loc[all_data_clean['year_sample']==2022]
variable_list= ['Total Housing Units', 'Total Housing Units: Occupied', 
    'Total Housing Units: Vacant', 'Vacant Housing Units: Seasonal, recreational, or occasional use',
    'Occupied Housing Units: Owner Occupied', 'Occupied Housing Units: Renter Occupied', 'Total Population']
summary_data_2022 = sum_multiple_variables(all_data_clean_2022,variable_list,'Update Categories')
summary_data_2022.to_excel('summary_data_2022.xlsx')

## Median Estimates

In [ ]:
grouping_variables = ['variable_category', 'year_sample', 'dataset', 'variable_code', 'sample_level', 'variable_name']

clean_income_data= all_data_clean.loc[(all_data_clean['county']!='510')]
clean_income_data = clean_income_data.loc[(clean_income_data['value']>=0)]
bin_column= 'variable_name'
sort_column = 'variable_code'
count_column = 'value'
category_field = 'variable_category'
category = 'Household Income'
cumulative_sort_columns = ['year_sample', 'sample_level', 'variable_code']
median_income_basin = calculate_median_value(clean_income_data, bin_column, sort_column, count_column, category_field, category,grouping_variables, cumulative_sort_columns)

In [ ]:
def median_across_levels(df, bin_column, sort_column, count_column, category_field, category,grouping_variables, cumulative_sorting_variables):
    basin_sort_columns = cumulative_sorting_variables
    basin_grouping_variables = grouping_variables
    county_sort_columns = ['county_name'] + cumulative_sorting_variables
    county_grouping_variables = grouping_variables + ['county_name']
    north_south_sort_columns = ['north_south'] + cumulative_sorting_variables
    north_south_grouping_variables = grouping_variables + ['north_south']
    basin_summary = calculate_median_value(df, bin_column, sort_column, count_column, category_field, category,grouping_variables, cumulative_sorting_variables)
    county_summary = calculate_median_value(df, bin_column, sort_column, count_column, category_field, category, county_grouping_variables, county_sort_columns)
    north_south_summary = calculate_median_value(df, bin_column, sort_column, count_column, category_field, category, north_south_grouping_variables, north_south_sort_columns)
    basin_summary['Geography'] = 'Basin'
    county_summary['Geography'] = county_summary['county_name']
    north_south_summary['Geography'] = north_south_summary['north_south']
    columns_to_keep = ['variable_code','variable_name', 'value', 'Geography', 'year_sample', 'dataset', 'sample_level', 'median_value']
    basin_summary= basin_summary[columns_to_keep]
    county_summary = county_summary[columns_to_keep]
    north_south_summary = north_south_summary[columns_to_keep]
    combined_summary = pd.concat([basin_summary, county_summary, north_south_summary], ignore_index=True)
    combined_summary['Category'] = category
    return combined_summary

grouping_variables = ['variable_category', 'year_sample', 'dataset', 'variable_code', 'variable_name', 'sample_level']

clean_income_data= all_data_clean.loc[(all_data_clean['county']!='510')&(all_data_clean['sample_level']=='tract')]

clean_income_data = clean_income_data.loc[(clean_income_data['value']>=0)]
bin_column= 'variable_name'
sort_column = 'variable_code'
count_column = 'value'
category_field = 'variable_category'
category = 'Household Income'
cumulative_sort_columns = ['year_sample', 'variable_code', 'sample_level']
median_incomes = median_across_levels(clean_income_data, bin_column, sort_column, count_column, category_field, category,grouping_variables, cumulative_sort_columns)

In [ ]:
median_incomes.to_excel("Updated_Median_Incomes.xlsx")

In [ ]:
clean_income_data= all_data_clean_2022

clean_income_data = clean_income_data.loc[(clean_income_data['value']>=0)]
median_incomes = median_across_levels(clean_income_data, bin_column, sort_column, count_column, category_field, category,grouping_variables, cumulative_sort_columns)

# Categorize Values

## Categorize Tahoe Age data

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
census_data = pd.DataFrame([feature.attributes for feature in feature_list])
age_data = census_data.loc[(census_data['variable_category']=='Age')&(census_data['sample_level']=='tract')]

In [ ]:
dec_categories_1990 = 'Census_Category_Lists/age_categories_1990.csv'
dec_categories = 'Census_Category_Lists/dec_age_categories.csv'
dec_categories_2020 = 'Census_Category_Lists/dec_age_categories_2020.csv'
tahoe_age_1990 = age_data.loc[age_data['year_sample']==1990]
tahoe_age_2010 = age_data.loc[age_data['year_sample']==2010]
tahoe_age_2000 = age_data.loc[age_data['year_sample']==2000]
tahoe_age_2020 = age_data.loc[age_data['year_sample']==2020]

grouped_tahoe_age_1990 = categorize_values(tahoe_age_1990, dec_categories_1990 ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')
grouped_tahoe_age_2000 = categorize_values(tahoe_age_2000, dec_categories ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')
grouped_tahoe_age_2010 = categorize_values(tahoe_age_2010, dec_categories ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')
grouped_tahoe_age_2020 = categorize_values(tahoe_age_2020, dec_categories_2020 ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')

grouped_tahoe_age = pd.concat([grouped_tahoe_age_1990, grouped_tahoe_age_2000, grouped_tahoe_age_2010, grouped_tahoe_age_2020], ignore_index=True)
grouped_tahoe_age.to_excel('Census_Data_Downloads\grouped_tahoe_age_10_year.xlsx')

In [ ]:
acs_categories_2022 = 'Census_Category_Lists/acs_age_groups.csv'
age_data_2022 = age_data.loc[age_data['year_sample']==2022]
grouped_tahoe_age_2022 = categorize_values(age_data_2022,acs_categories_2022,'Broad_Category', 'TRPA 5-year Age Categories')
grouped_tahoe_age_2022.to_excel('Census_Data_Downloads\grouped_acs_age_5_year.xlsx')

## Categorize non-Tahoe age data

In [ ]:
#Import from an excel sheet because it's not worth bringing the detailed ones in
dtypes = {
    'GEO_ID' : str,
'value' : int,
'state' : str,
'MarginOfError' : str,
'county' : str,
'sample_level' : str,
'Geo_Name' : str,
'variable_code' : str,
'variable_name' : str,
'variable_category' : str,
'year_sample' : str,
'dataset' : str,
'census_geom_year' : str,
'TRPAID' : str,
}

non_tahoe_age = pd.read_csv("non_tahoe_age_data.csv", dtype= dtypes)

In [ ]:
dec_categories = 'Census_Category_Lists/dec_age_categories.csv'
dec_categories_2020 = 'Census_Category_Lists/dec_age_categories_2020.csv'
non_tahoe_age_2010 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2010']
non_tahoe_age_2000 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2000']
non_tahoe_age_2020 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2020']

grouped_non_tahoe_age_2000 = categorize_values(non_tahoe_age_2000, dec_categories ,'Census Category with Sex', 'TRPA Census Age Sex Categories Grouped: ')
grouped_non_tahoe_age_2010 = categorize_values(non_tahoe_age_2010, dec_categories ,'Census Category with Sex', 'TRPA Census Age Sex Categories Grouped: ')
grouped_non_tahoe_age_2020 = categorize_values(non_tahoe_age_2020, dec_categories_2020 ,'Census Category with Sex', 'TRPA Census Age Sex Categories Grouped: ')

grouped_non_tahoe_age = pd.concat([grouped_non_tahoe_age_2000, grouped_non_tahoe_age_2010, grouped_non_tahoe_age_2020], ignore_index=True)
grouped_non_tahoe_age.to_excel('Census_Data_Downloads\grouped_non_tahoe_age.xlsx')

In [ ]:
dec_categories = 'Census_Category_Lists/dec_age_categories.csv'
dec_categories_2020 = 'Census_Category_Lists/dec_age_categories_2020.csv'
non_tahoe_age_2010 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2010']
non_tahoe_age_2000 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2000']
non_tahoe_age_2020 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2020']

grouped_non_tahoe_age_2000 = categorize_values(non_tahoe_age_2000, dec_categories ,'Census Category', 'TRPA Census Age Categories Grouped: ')
grouped_non_tahoe_age_2010 = categorize_values(non_tahoe_age_2010, dec_categories ,'Census Category', 'TRPA Census Age Categories Grouped: ')
grouped_non_tahoe_age_2020 = categorize_values(non_tahoe_age_2020, dec_categories_2020 ,'Census Category', 'TRPA Census Age Categories Grouped: ')

grouped_non_tahoe_age = pd.concat([grouped_non_tahoe_age_2000, grouped_non_tahoe_age_2010, grouped_non_tahoe_age_2020], ignore_index=True)
grouped_non_tahoe_age.to_excel('Census_Data_Downloads\grouped_non_tahoe_age_no_sex.xlsx')

In [ ]:
dec_categories = 'Census_Category_Lists/dec_age_categories.csv'
dec_categories_2020 = 'Census_Category_Lists/dec_age_categories_2020.csv'
non_tahoe_age_2010 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2010']
non_tahoe_age_2000 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2000']
non_tahoe_age_2020 = non_tahoe_age.loc[non_tahoe_age['year_sample']=='2020']

grouped_non_tahoe_age_2000 = categorize_values(non_tahoe_age_2000, dec_categories ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')
grouped_non_tahoe_age_2010 = categorize_values(non_tahoe_age_2010, dec_categories ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')
grouped_non_tahoe_age_2020 = categorize_values(non_tahoe_age_2020, dec_categories_2020 ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')

grouped_non_tahoe_age = pd.concat([grouped_non_tahoe_age_2000, grouped_non_tahoe_age_2010, grouped_non_tahoe_age_2020], ignore_index=True)
grouped_non_tahoe_age.to_excel('Census_Data_Downloads\grouped_non_tahoe_age_10_year.xlsx')

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
census_data = pd.DataFrame([feature.attributes for feature in feature_list])
age_data_1990 = census_data.loc[(census_data['variable_category']=='TRPA Census Age Categories Grouped: Age')&(census_data['sample_level']=='County')
                                & (census_data['year_sample']==1990)]

In [ ]:
dec_categories_1990 = 'Census_Category_Lists/age_categories_1990_updated.csv'
grouped_tahoe_age_1990 = categorize_values(age_data_1990, dec_categories_1990 ,'Ten Year Category', 'TRPA 10-year Age Categories Grouped: ')

In [ ]:
grouped_tahoe_age_1990.to_excel('grouped_tahoe_age_1990_2.xlsx')

## Categorize Vehicle Data

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
census_data = pd.DataFrame([feature.attributes for feature in feature_list])
vehicle_data = census_data.loc[(census_data['variable_category']=='Tenure by Vehicles Available')]

In [ ]:
vehicle_categories = 'Census_Category_Lists/vehicle_categories.csv'
grouped_vehicle_data = categorize_values(vehicle_data, vehicle_categories, 'Broad Category', 'Household Vehicle Numbers: ')

In [ ]:
grouped_vehicle_data.to_excel('Census_Data_Downloads/vehicle_data.xlsx')

#### Get 1990 Miscellanous vacant

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
census_data = pd.DataFrame([feature.attributes for feature in feature_list])
housing_data = census_data.loc[census_data['variable_name'].isin(['Total Housing Units: Vacant', 'Vacant Housing Units: Seasonal, recreational, or occasional use'])]

In [ ]:
def vacant_other(group):
    x_value = group.loc[group['variable_name']=='Total Housing Units: Vacant','value'].values[0]
    y_value = group.loc[group['variable_name']=='Vacant Housing Units: Seasonal, recreational, or occasional use','value'].values[0]
    return x_value - y_value
housing_data = census_data.loc[census_data['variable_name'].isin(['Total Housing Units: Vacant', 'Vacant Housing Units: Seasonal, recreational, or occasional use'])]
housing_data =housing_data.loc[housing_data['year_sample']==1990]

columns_to_exclude = ['variable_name', 'value', 'variable_code', 'OBJECTID']

# Use list comprehension to get all column names except the ones to exclude
selected_columns = [col for col in housing_data.columns if col not in columns_to_exclude]


housing_data['Difference'] = housing_data.groupby(selected_columns).apply(vacant_other).reset_index(drop=True)



## Categorize Language Data

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
census_data = pd.DataFrame([feature.attributes for feature in feature_list])
language_data = census_data.loc[(census_data['variable_category']=='Language')]

In [ ]:
language_categories = 'Census_Category_Lists/Language_Categories.csv'
grouped_language_data = categorize_values(language_data, language_categories, 'English Fluency', 'English Fluency: ')
grouped_language_data = grouped_language_data.dropna(subset=['variable_name'])

In [ ]:
grouped_language_data.to_excel('Census_Data_Downloads/language_data.xlsx')